# MNISTのデータを扱うための前準備

In [1]:
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    #MNISTデータをpathからロード。引数に指定したパスを結合（ラベルや画像のパスを作成）
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)
    
    #ファイルを読み込む
    #引数にファイル、モードを指定　rb（read binary）は読み込みのバイナリモード）
    with open(labels_path, 'rb') as lbpath:
        #バイナリを文字列に変換：unpack関数の引数にフォーマット、8バイト分の
        #バイナリデータを指定してマジックナンバー、アイテムの個数を読み込む
        magic, n = struct.unpack('>II', lbpath.read(8))
        #ファイルからラベルを読み込み配列を構築：fromfile関数の引数に
        #ファイル、配列のデータ形式を指定
        labels = np.fromfile(lbpath, dtype=np.uint8)
    
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols =struct.unpack(">IIII", imgpath.read(16))
        #画像ピクセル情報の配列のサイズを変更
        #（行数：ラベルのサイズ、数列：特徴量の個数）
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
    return images, labels

# 前処理

In [2]:
x_train, y_train = load_mnist('.', kind='train')
print('Rows: %d, Columns: %d' %(x_train.shape[0],x_train.shape[1]))
x_test, y_test = load_mnist('.' ,kind='t10k')
print('Rows: %d, Columns: %d' %(x_test.shape[0],x_test.shape[1]))

Rows: 60000, Columns: 784
Rows: 10000, Columns: 784


In [3]:
#標準化、正規化
mean_vals = np.mean(x_train, axis=0)
std_val = np.std(x_train)

x_train_cntered = (x_train - mean_vals) / std_val
x_test_centered = (x_test - mean_vals) / std_val
del x_train, x_test
print(x_train_cntered.shape, y_train.shape)
print(x_test_centered.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


# ~Tensorflow~

# 計算グラフの作成

In [4]:
import tensorflow as tf

n_features = x_train_cntered.shape[1]
n_classes = 10
random_seed =123
np.random.seed(random_seed)

#計算グラフの作成開始
g = tf.Graph()
with g.as_default():
    tf.random.set_seed(random_seed)
    tf_x = tf.placeholder(dtype=tf.float32,
                       shape=(None, n_features),
                       name='tf_x')

    tf_y = tf.placeholder(dtype=tf.int32, 
                        shape=None, name='tf_y')
    y_onehot = tf.one_hot(indices=tf_y, depth=n_classes)

    h1 = tf.layers.dense(inputs=tf_x, units=50,
                         activation=tf.tanh,
                         name='layer1')

    h2 = tf.layers.dense(inputs=h1, units=50,
                         activation=tf.tanh,
                         name='layer2')

    logits = tf.layers.dense(inputs=h2, 
                             units=10,
                             activation=None,
                             name='layer3')

    predictions = {
        'classes' : tf.argmax(logits, axis=1, 
                              name='predicted_classes'),
        'probabilities' : tf.nn.softmax(logits, 
                              name='softmax_tensor')
    }

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [5]:
#コスト関数とオプティマイザを定義
with g.as_default():
    cost = tf.losses.softmax_cross_entropy(
            onehot_labels=y_onehot, logits=logits)

    optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=0.001)

    train_op = optimizer.minimize(loss=cost)
#モデルの変数とオプティマイザを初期化するための演算子を定義
    init_op = tf.global_variables_initializer()

AttributeError: module 'tensorflow.keras.losses' has no attribute 'softmax_cross_entropy'

In [6]:
#データバッチ用のジェネレータを返す関数を実装
def create_batch_generator(X, y, batch_size=128, shuffle=False):
    X_copy = np.array(X)
    y_copy = np.array(y)
    
    if shuffle:
        data = np.column_stack((X_copy, y_copy))
        np.random.shuffle(data)
        X_copy = data[:, :-1]
        y_copy = data[:, -1].astype(int)
    
    for i in range(0, X.shape[0], batch_size):
        yield (X_copy[i:i+batch_size, :], y_copy[i:i+batch_size])

# セッションを開始して実際に計算グラフを実行

In [7]:
## 計算グラフを起動するためのセッションを作成
sess =  tf.Session(graph=g)
## 変数イニシャライザを実行
sess.run(init_op)

## 50 エッポクのトレーニング
training_costs = []
for epoch in range(50):
    training_loss = []
    batch_generator = create_batch_generator(
            X_train_centered, y_train, 
            batch_size=64)
    for batch_X, batch_y in batch_generator:
        ## prepare a dict to feed data to our network:
        feed = {tf_x:batch_X, tf_y:batch_y}
        _, batch_cost = sess.run([train_op, cost],
                                 feed_dict=feed)
        training_costs.append(batch_cost)
    print(' -- Epoch %2d  '
          'Avg. Training Loss: %.4f' % (
              epoch+1, np.mean(training_costs)
    ))

AttributeError: module 'tensorflow' has no attribute 'Session'

# 上記でトレーニング済みのモデルをテストデータセットで使用して予測

In [8]:
feed = {tf_x : x_test_centered}
y_pred = sees.run(predictions['classes'], feed_dict=feed)

print('Test Accuracy: %2f%%' %
     (100 * np.sum(y_pred ==y_test) / y_test.shape[0]))

NameError: name 'tf_x' is not defined

In [9]:
import tensorflow as tf
print(tf.__version__)

2.3.1
